## Simulations

In [2]:
import numpy as np
import rasterio
from rasterio.plot import reshape_as_image, reshape_as_raster
from scipy.ndimage import shift, rotate
import time

start_time = time.time()

# Input path
image_path = r"D:\Yuvraj\MSI\bbr\data\200006033301_01\200006033301_01\200006033301_01_P001_MUL\24SEP27155103-M2AS-200006033301_01_P001.TIF"

# Open image
with rasterio.open(image_path) as src:
    bands = src.count
    profile = src.profile
    data = src.read()  # shape: (bands, height, width)

print(f"Total bands in image: {bands}")

# Reference band
ref_band = 3  # keep Band 3 fixed

# -------------------------
# 1. Translations only
# -------------------------
# Define band-specific shifts (dy, dx) in pixels
# -------------------------
# 1. Translations only
# -------------------------
# Define band-specific shifts (dy, dx) in pixels (50–150 px range)
translations = {
    1: (60, -120),   # Band 1 shifted down 60, left 120
    2: (90, 100),    # Band 2 shifted down 90, right 100
    3: (0, 0),       # Ref band fixed
    4: (-70, 130),   # Band 4 shifted up 70, right 130
    5: (140, -80),   # Band 5 shifted down 140, left 80
    6: (-110, -60),  # Band 6 shifted up 110, left 60
    7: (150, 90),    # Band 7 shifted down 150, right 90
    8: (-100, -140)  # Band 8 shifted up 100, left 140
}

translated_data = np.zeros_like(data)
for b in range(1, bands+1):
    dy, dx = translations[b]
    translated_data[b-1] = shift(data[b-1], shift=(dy, dx), mode="nearest")

# -------------------------
# 2. Translations + Rotations
# -------------------------
# Define band-specific (dy, dx, angle in degrees)
transforms = {
    1: (60, -120, 0.5),    # Band 1
    2: (90, 100, -1.0),    # Band 2
    3: (0, 0, 0.0),        # Ref band fixed
    4: (-70, 130, 0.8),    # Band 4
    5: (140, -80, -0.6),   # Band 5
    6: (-110, -60, 1.2),   # Band 6
    7: (150, 90, -0.7),    # Band 7
    8: (-100, -140, 0.4)   # Band 8
}


transformed_data = np.zeros_like(data)
for b in range(1, bands+1):
    dy, dx, ang = transforms[b]
    shifted = shift(data[b-1], shift=(dy, dx), mode="nearest")
    transformed_data[b-1] = rotate(shifted, angle=ang, reshape=False, mode="nearest")

# -------------------------
# Save results
# -------------------------
out1 = image_path.replace(".TIF", "_simulated_translations.tif")
out2 = image_path.replace(".TIF", "_simulated_trans_and_rotation.tif")

with rasterio.open(out1, "w", **profile) as dst:
    dst.write(translated_data)

with rasterio.open(out2, "w", **profile) as dst:
    dst.write(transformed_data)

print(f"Saved simulated images:\n  {out1}\n  {out2}")

end_time = time.time()
elapsed = end_time - start_time
# Convert to minutes and seconds
mins, secs = divmod(elapsed, 60)
print(f"⏱️ Process completed in {int(mins)} min {secs:.2f} sec")

Total bands in image: 8
Saved simulated images:
  D:\Yuvraj\MSI\bbr\data\200006033301_01\200006033301_01\200006033301_01_P001_MUL\24SEP27155103-M2AS-200006033301_01_P001_simulated_translations.tif
  D:\Yuvraj\MSI\bbr\data\200006033301_01\200006033301_01\200006033301_01_P001_MUL\24SEP27155103-M2AS-200006033301_01_P001_simulated_trans_and_rotation.tif
⏱️ Process completed in 0 min 44.50 sec


In [3]:
# Simulation: translation

In [1]:
import numpy as np
import rasterio
from skimage.transform import ProjectiveTransform, warp
from skimage.transform import AffineTransform, warp
from extract_sift_features import extract_sift_features
from sklearn.linear_model import RANSACRegressor
from sklearn.metrics import mean_squared_error
import os

import sys
#sys.path.append("utils")  # or full path to folder
from ransacgpu import RANSACRegressorGPU

import time
start_time = time.time()

#########################

# -------------------------
# INPUT PARAMETERS
# -------------------------
image_patch = r"D:\Yuvraj\MSI\bbr\data\200006033301_01\200006033301_01\200006033301_01_P001_MUL\24SEP27155103-M2AS-200006033301_01_P001_simulated_translations.tif"
output_path = r"D:\Yuvraj\MSI\bbr\data\200006033301_01\200006033301_01\200006033301_01_P001_MUL\sim_translation_registered_stack_result.tif"
ref_band = 3

# SIFT parameters
sift_params = {
    "image_row": 5,
    "image_col": 5,
    "nfeatures": 130,
    "nOctaveLayers": 3,
    "contrastThreshold": 0.01,
    "edgeThreshold": 15,
    "sigma": 1.2,
    "eightbit_stretch": False,
    "sobel_filter": False,
    "return_keypoints": True,
    "return_descriptors": True,
    "return_coords": True,
    "display_figure": False
}

# -------------------------
# Load image and metadata
# -------------------------
with rasterio.open(image_patch) as src:
    image_data = src.read()  # shape: (bands, H, W)
    image_profile = src.profile
    total_bands = src.count
    input_dtype = src.dtypes[0]

print(f"[INFO] Total bands: {total_bands}, dtype: {input_dtype}")

# Initialize band stack with reference band
band_stack = np.zeros_like(image_data, dtype=np.float32)
band_stack[ref_band - 1] = image_data[ref_band - 1]

# Get other bands dynamically
other_bands = [b for b in range(1, total_bands + 1) if b != ref_band]

# -------------------------
# Step 1: Extract features from reference band
# -------------------------
print(f"[INFO] Extracting features from reference band {ref_band}...")
block_info_ref, all_kps_ref, all_desc_ref, all_coords_ref = extract_sift_features(
    image_patch, band_number=ref_band, **sift_params
)
print(f"[INFO] Reference band features extracted. Blocks: {len(block_info_ref)}")

# -------------------------
# Helper function: Align keypoints block-wise
# -------------------------
def align_block_keypoints(block_info_image1, block_info_image2):
    strong_points_image1_sorted = []
    strong_points_image2_sorted = []
    strong_points_image1_sorted_response = []
    strong_points_image2_sorted_response = []

    strong_block_info_image1 = {}
    strong_block_info_image2 = {}

    for block_number in block_info_image1:
        if block_number in block_info_image2:
            if block_info_image1[block_number]['status'] == 1 and block_info_image2[block_number]['status'] == 1:
                num1 = block_info_image1[block_number]['num_points']
                num2 = block_info_image2[block_number]['num_points']
                min_points = min(num1, num2)

                # Take top points by response
                kps1_sorted = sorted(block_info_image1[block_number]['keypoints'], key=lambda x: x['response'], reverse=True)[:min_points]
                kps2_sorted = sorted(block_info_image2[block_number]['keypoints'], key=lambda x: x['response'], reverse=True)[:min_points]

                strong_block_info_image1[block_number] = {'block_number': block_number, 'status': 1, 'num_points': min_points, 'keypoints': kps1_sorted}
                strong_block_info_image2[block_number] = {'block_number': block_number, 'status': 1, 'num_points': min_points, 'keypoints': kps2_sorted}

                for kp in kps1_sorted:
                    strong_points_image1_sorted.append(kp['coord'])
                    strong_points_image1_sorted_response.append(kp['response'])
                for kp in kps2_sorted:
                    strong_points_image2_sorted.append(kp['coord'])
                    strong_points_image2_sorted_response.append(kp['response'])

    return (
        np.array(strong_points_image1_sorted),
        np.array(strong_points_image2_sorted),
        np.array(strong_points_image1_sorted_response),
        np.array(strong_points_image2_sorted_response),
        strong_block_info_image1,
        strong_block_info_image2
    )

# -------------------------
# Step 2: Process and register all other bands
# -------------------------
for band in other_bands:
    print(f"\n[INFO] Processing band {band}...")

    # Extract features for this band
    block_info_band, all_kps_band, all_desc_band, all_coords_band = extract_sift_features(
        image_patch, band_number=band, **sift_params
    )

    # Align keypoints block-wise
    strong_pts_ref, strong_pts_band, resp_ref, resp_band, strong_block_ref, strong_block_band = align_block_keypoints(block_info_ref, block_info_band)
    print(f"[INFO] Band {band}_ - Total candidate matches: {len(strong_pts_ref)}")


    # --------------------------------------------------
    # MSAC
    # --------------------------------------------------

    # Define the MSAC-based RANSAC model

    msac_model = RANSACRegressorGPU(
        min_samples=3,
        residual_threshold=19,  # Adjusted to handle expected noise scale
        max_trials=80000,  # Increased for robustness in large datasets
        stop_n_inliers=len(strong_pts_ref),
        loss='absolute_error',  # Robust against potential outliers
        random_state=42,  # Ensures reproducibility
    )
    # Fit the model
    msac_model.fit(strong_pts_ref, strong_pts_band)
    #predicted_pts_band = msac_model.predict(strong_pts_ref)
    inlier_mask = msac_model.inlier_mask_
    outlier_mask = ~inlier_mask

    inliers_ref = strong_pts_ref[inlier_mask]
    inliers_band = strong_pts_band[inlier_mask]
    inliers_ref_resp = resp_ref[inlier_mask]
    inliers_band_resp = resp_band[inlier_mask]

    # RMSE for inliers
    rmse = np.sqrt(mean_squared_error(inliers_ref, inliers_band))
    #print(f"[INFO] Band {band} - Number of inliers: {len(inliers_ref)}, RMSE: {rmse:.3f}")
    print(f"[INFO] Band {band} - Number of inliers: {len(inliers_ref)}")

    affine_tf = AffineTransform()
    affine_tf.estimate(inliers_band, inliers_ref)  # Warp band → ref

    # Print affine transformation matrix in a readable form
    print("[INFO] Affine Transformation Matrix (band → reference):")
    for row in affine_tf.params:
        print("  ".join(f"{val:.8f}" for val in row))
    # Decompose
    from affine_utils import decompose_affine
    params = decompose_affine(affine_tf)
    print(params)  # You can also access dict values

    # RMSE after affine transform
    warped_pts = affine_tf(inliers_band)
    rmse_transformed = np.sqrt(mean_squared_error(warped_pts, inliers_ref))
    print(f"[INFO] Band {band} - RMSE after affine transform: {rmse_transformed:.3f}")

    # Warp the band to reference
    warped_band = warp(image_data[band-1], inverse_map=affine_tf.inverse, output_shape=image_data[0].shape, order=3)
    band_stack[band-1] = warped_band.astype(np.float32)


# -------------------------
# Step 3: Export band_stack as GeoTIFF
# -------------------------
# Update metadata
image_profile.update({
    "count": total_bands,
    "dtype": np.float32
})

with rasterio.open(output_path, "w", **image_profile) as dst:
    dst.write(band_stack)

print(f"\n✅ Registered band stack saved at: {output_path}")

end_time = time.time()
elapsed = end_time - start_time
# Convert to minutes and seconds
mins, secs = divmod(elapsed, 60)
print(f"⏱️ Process completed in {int(mins)} min {secs:.2f} sec")


Package Version: 2.0.0a0
[INFO] Total bands: 8, dtype: uint8
[INFO] Extracting features from reference band 3...
[INFO] Loaded band 3 from D:\Yuvraj\MSI\bbr\data\200006033301_01\200006033301_01\200006033301_01_P001_MUL\24SEP27155103-M2AS-200006033301_01_P001_simulated_translations.tif, shape=(5102, 5189), dtype=uint8
[INFO] SIFT detector initialized
[INFO] Dividing image into 5 x 5 blocks of approx 1020 x 1037 pixels
[INFO] Feature extraction completed
[INFO] Reference band features extracted. Blocks: 25

[INFO] Processing band 1...
[INFO] Loaded band 1 from D:\Yuvraj\MSI\bbr\data\200006033301_01\200006033301_01\200006033301_01_P001_MUL\24SEP27155103-M2AS-200006033301_01_P001_simulated_translations.tif, shape=(5102, 5189), dtype=uint8
[INFO] SIFT detector initialized
[INFO] Dividing image into 5 x 5 blocks of approx 1020 x 1037 pixels
[INFO] Feature extraction completed
[INFO] Band 1_ - Total candidate matches: 3250


2025/11/05 13:00:28 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.


runnning optimised version for ransac on self.device='cuda'
[INFO] Band 1 - Number of inliers: 85
[INFO] Affine Transformation Matrix (band → reference):
1.00044001  -0.00019662  119.33156628
-0.00093110  0.99927903  -54.77889065
0.00000000  0.00000000  1.00000000
[INFO] Affine Transformation Matrix:
[[1.00044001 -0.00019662 119.33156628]
 [-0.00093110 0.99927903 -54.77889065]
 [0.00000000 0.00000000 1.00000000]]
[INFO] Decomposed Parameters:
  Translation : (tx=119.33156628, ty=-54.77889065)
  Rotation    : -0.05332462°
  Scale       : (sx=1.00044044, sy=0.99927905)
  Shear       : 0.06459838°
{'translation': (np.float64(119.33156627824383), np.float64(-54.778890651580696)), 'rotation_deg': np.float64(-0.05332462460546274), 'scale': (np.float64(1.0004404431913043), np.float64(0.9992790466548206)), 'shear_deg': np.float64(0.06459838334658975)}
[INFO] Band 1 - RMSE after affine transform: 3.490

[INFO] Processing band 2...
[INFO] Loaded band 2 from D:\Yuvraj\MSI\bbr\data\200006033301_01

[INFO] Total bands: 8, dtype: uint8
[INFO] Extracting features from reference band 3...
[INFO] Loaded band 3 from D:\Yuvraj\MSI\bbr\data\200006033301_01\200006033301_01\200006033301_01_P001_MUL\24SEP27155103-M2AS-200006033301_01_P001_simulated_translations.tif, shape=(5102, 5189), dtype=uint8
[INFO] SIFT detector initialized
[INFO] Dividing image into 5 x 5 blocks of approx 1020 x 1037 pixels
[INFO] Feature extraction completed
[INFO] Reference band features extracted. Blocks: 25

[INFO] Processing band 1...
[INFO] Loaded band 1 from D:\Yuvraj\MSI\bbr\data\200006033301_01\200006033301_01\200006033301_01_P001_MUL\24SEP27155103-M2AS-200006033301_01_P001_simulated_translations.tif, shape=(5102, 5189), dtype=uint8
[INFO] SIFT detector initialized
[INFO] Dividing image into 5 x 5 blocks of approx 1020 x 1037 pixels
[INFO] Feature extraction completed
runnning optimised version for ransac on self.device='cuda'
[INFO] Band 1 - Number of inliers: 72
[INFO] Band 1 with ref band - RMSE in pixels

In [ ]:
# sim translations : RMSE calculation

In [1]:
import numpy as np
import rasterio
from skimage.transform import ProjectiveTransform, warp
from skimage.transform import AffineTransform, warp
from extract_sift_features import extract_sift_features
from sklearn.linear_model import RANSACRegressor
from sklearn.metrics import mean_squared_error
import os
import time
start_time = time.time()

#########################

# -------------------------
# INPUT PARAMETERS
# -------------------------
image_patch = r"D:\Yuvraj\MSI\bbr\data\200006033301_01\200006033301_01\200006033301_01_P001_MUL\24SEP27155103-M2AS-200006033301_01_P001_simulated_translations.tif"
output_path = r"D:\Yuvraj\MSI\bbr\data\200006033301_01\200006033301_01\200006033301_01_P001_MUL\sim_translation_registered_stack_result.tif"
ref_band = 3

# SIFT parameters
sift_params = {
    "image_row": 5,
    "image_col": 5,
    "nfeatures": 100,
    "nOctaveLayers": 3,
    "contrastThreshold": 0.01,
    "edgeThreshold": 15,
    "sigma": 1.2,
    "eightbit_stretch": False,
    "sobel_filter": False,
    "return_keypoints": True,
    "return_descriptors": True,
    "return_coords": True,
    "display_figure": False
}

# -------------------------
# Load image and metadata
# -------------------------
with rasterio.open(image_patch) as src:
    image_data = src.read()  # shape: (bands, H, W)
    image_profile = src.profile
    total_bands = src.count
    input_dtype = src.dtypes[0]

print(f"[INFO] Total bands: {total_bands}, dtype: {input_dtype}")

# Initialize band stack with reference band
band_stack = np.zeros_like(image_data, dtype=np.float32)
band_stack[ref_band - 1] = image_data[ref_band - 1]

# Get other bands dynamically
other_bands = [b for b in range(1, total_bands + 1) if b != ref_band]

# -------------------------
# Step 1: Extract features from reference band
# -------------------------
print(f"[INFO] Extracting features from reference band {ref_band}...")
block_info_ref, all_kps_ref, all_desc_ref, all_coords_ref = extract_sift_features(
    image_patch, band_number=ref_band, **sift_params
)
print(f"[INFO] Reference band features extracted. Blocks: {len(block_info_ref)}")

# -------------------------
# Helper function: Align keypoints block-wise
# -------------------------
def align_block_keypoints(block_info_image1, block_info_image2):
    strong_points_image1_sorted = []
    strong_points_image2_sorted = []
    strong_points_image1_sorted_response = []
    strong_points_image2_sorted_response = []

    strong_block_info_image1 = {}
    strong_block_info_image2 = {}

    for block_number in block_info_image1:
        if block_number in block_info_image2:
            if block_info_image1[block_number]['status'] == 1 and block_info_image2[block_number]['status'] == 1:
                num1 = block_info_image1[block_number]['num_points']
                num2 = block_info_image2[block_number]['num_points']
                min_points = min(num1, num2)

                # Take top points by response
                kps1_sorted = sorted(block_info_image1[block_number]['keypoints'], key=lambda x: x['response'], reverse=True)[:min_points]
                kps2_sorted = sorted(block_info_image2[block_number]['keypoints'], key=lambda x: x['response'], reverse=True)[:min_points]

                strong_block_info_image1[block_number] = {'block_number': block_number, 'status': 1, 'num_points': min_points, 'keypoints': kps1_sorted}
                strong_block_info_image2[block_number] = {'block_number': block_number, 'status': 1, 'num_points': min_points, 'keypoints': kps2_sorted}

                for kp in kps1_sorted:
                    strong_points_image1_sorted.append(kp['coord'])
                    strong_points_image1_sorted_response.append(kp['response'])
                for kp in kps2_sorted:
                    strong_points_image2_sorted.append(kp['coord'])
                    strong_points_image2_sorted_response.append(kp['response'])

    return (
        np.array(strong_points_image1_sorted),
        np.array(strong_points_image2_sorted),
        np.array(strong_points_image1_sorted_response),
        np.array(strong_points_image2_sorted_response),
        strong_block_info_image1,
        strong_block_info_image2
    )

# -------------------------
# Step 2: Process and register all other bands
# -------------------------
for band in other_bands:
    print(f"\n[INFO] Processing band {band}...")

    # Extract features for this band
    block_info_band, all_kps_band, all_desc_band, all_coords_band = extract_sift_features(
        image_patch, band_number=band, **sift_params
    )

    # Align keypoints block-wise
    strong_pts_ref, strong_pts_band, resp_ref, resp_band, strong_block_ref, strong_block_band = align_block_keypoints(block_info_ref, block_info_band)
    #print(f"[INFO] Band {band}_ - Total candidate matches: {len(strong_pts_ref)}")


    # # MSAC / RANSAC to get inliers
    # msac_model = RANSACRegressor(
    #     min_samples=3,
    #     residual_threshold=19,
    #     max_trials=80000,
    #     stop_n_inliers=len(strong_pts_ref),
    #     loss='absolute_error',
    #     random_state=42
    # )
    # msac_model.fit(strong_pts_ref, strong_pts_band)
    # predicted_pts_band = msac_model.predict(strong_pts_ref)
    # inlier_mask = msac_model.inlier_mask_
    # outlier_mask = ~inlier_mask

    # inliers_ref = strong_pts_ref[inlier_mask]
    # inliers_band = strong_pts_band[inlier_mask]
    # inliers_ref_resp = resp_ref[inlier_mask]
    # inliers_band_resp = resp_band[inlier_mask]

    import sys
    #sys.path.append("utils")  # or full path to folder
    from ransacgpu import RANSACRegressorGPU

    # --------------------------------------------------
    # MSAC
    # --------------------------------------------------

    # Define the MSAC-based RANSAC model

    msac_model = RANSACRegressorGPU(
        min_samples=3,
        residual_threshold=3,  # Adjusted to handle expected noise scale
        max_trials=10000,  # Increased for robustness in large datasets
        stop_n_inliers=len(strong_pts_ref),
        loss='absolute_error',  # Robust against potential outliers
        random_state=42,  # Ensures reproducibility
    )
    # Fit the model
    msac_model.fit(strong_pts_ref, strong_pts_band)
    #predicted_pts_band = msac_model.predict(strong_pts_ref)
    inlier_mask = msac_model.inlier_mask_
    outlier_mask = ~inlier_mask

    inliers_ref = strong_pts_ref[inlier_mask]
    inliers_band = strong_pts_band[inlier_mask]
    inliers_ref_resp = resp_ref[inlier_mask]
    inliers_band_resp = resp_band[inlier_mask]

    # RMSE for inliers
    rmse = np.sqrt(mean_squared_error(inliers_ref, inliers_band))
    #print(f"[INFO] Band {band} - Number of inliers: {len(inliers_ref)}, RMSE: {rmse:.3f}")
    print(f"[INFO] Band {band} - Number of inliers: {len(inliers_ref)}")

    # # Estimate projective transform (warp this band to reference)
    # projective_tf = ProjectiveTransform()
    # projective_tf.estimate(inliers_band, inliers_ref)  # Warp band → ref

    # # RMSE after transform
    # warped_pts = projective_tf(inliers_band)
    # rmse_transformed = np.sqrt(mean_squared_error(warped_pts, inliers_ref))
    # print(f"[INFO] Band {band} - RMSE after projective transform: {rmse_transformed:.3f}")

    # # Warp the band to reference
    # warped_band = warp(image_data[band-1], projective_tf.inverse, output_shape=image_data[0].shape, order=3)
    # band_stack[band-1] = warped_band.astype(np.float32)

    # Estimate affine transform (warp this band to reference)
    affine_tf = AffineTransform()
    affine_tf.estimate(inliers_band, inliers_ref)  # Warp band → ref

    # # Print affine transformation matrix in a readable form
    # print("[INFO] Affine Transformation Matrix (band → reference):")
    # for row in affine_tf.params:
    #     print("  ".join(f"{val:.8f}" for val in row))
    # # Decompose
    # from affine_utils import decompose_affine
    # params = decompose_affine(affine_tf)
    # print(params)  # You can also access dict values

# # --- 1. RMSE on all points (before transform) ---
#     rmse_all_raw = np.sqrt(mean_squared_error(strong_pts_band, strong_pts_ref))
#     rmse_x_raw = np.sqrt(np.mean((strong_pts_band[:,0] - strong_pts_ref[:,0])**2))
#     rmse_y_raw = np.sqrt(np.mean((strong_pts_band[:,1] - strong_pts_ref[:,1])**2))

    # print(f"[INFO] Band {band} - RMSE all points (raw, 2D): {rmse_all_raw:.3f}")
    # print(f"[INFO] Band {band} - RMSE all points (raw, X):  {rmse_x_raw:.3f}")
    # print(f"[INFO] Band {band} - RMSE all points (raw, Y):  {rmse_y_raw:.3f}")

    # 2. RMSE on affine-transformed inliers
    warped_inliers = affine_tf(inliers_band)
    rmse_inliers = np.sqrt(mean_squared_error(warped_inliers, inliers_ref))
    print(f"[INFO] Band {band} with ref band - RMSE in pixels): {rmse_inliers:.3f}")

    # # 3. RMSE on affine-transformed all points
    # warped_all = affine_tf(strong_pts_band)
    # rmse_all_transformed = np.sqrt(mean_squared_error(warped_all, strong_pts_ref))
    # print(f"[INFO] Band {band} - RMSE on affine-transformed all points: {rmse_all_transformed:.3f}")

    # # Warp the band to reference
    # warped_band = warp(image_data[band-1], inverse_map=affine_tf.inverse, output_shape=image_data[0].shape, order=3)
    # band_stack[band-1] = warped_band.astype(np.float32)


# # -------------------------
# # Step 3: Export band_stack as GeoTIFF
# # -------------------------
# # Update metadata
# image_profile.update({
#     "count": total_bands,
#     "dtype": np.float32
# })

# with rasterio.open(output_path, "w", **image_profile) as dst:
#     dst.write(band_stack)

# print(f"\n✅ Registered band stack saved at: {output_path}")
end_time = time.time()
elapsed = end_time - start_time
# Convert to minutes and seconds
mins, secs = divmod(elapsed, 60)
print(f"⏱️ Process completed in {int(mins)} min {secs:.2f} sec")

[INFO] Total bands: 8, dtype: uint8
[INFO] Extracting features from reference band 3...
[INFO] Loaded band 3 from D:\Yuvraj\MSI\bbr\data\200006033301_01\200006033301_01\200006033301_01_P001_MUL\24SEP27155103-M2AS-200006033301_01_P001_simulated_translations.tif, shape=(5102, 5189), dtype=uint8
[INFO] SIFT detector initialized
[INFO] Dividing image into 5 x 5 blocks of approx 1020 x 1037 pixels
[INFO] Feature extraction completed
[INFO] Reference band features extracted. Blocks: 25

[INFO] Processing band 1...
[INFO] Loaded band 1 from D:\Yuvraj\MSI\bbr\data\200006033301_01\200006033301_01\200006033301_01_P001_MUL\24SEP27155103-M2AS-200006033301_01_P001_simulated_translations.tif, shape=(5102, 5189), dtype=uint8
[INFO] SIFT detector initialized
[INFO] Dividing image into 5 x 5 blocks of approx 1020 x 1037 pixels
[INFO] Feature extraction completed
Package Version: 2.0.0a0


2025/11/05 13:05:02 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.


runnning optimised version for ransac on self.device='cuda'
[INFO] Band 1 - Number of inliers: 72
[INFO] Band 1 with ref band - RMSE in pixels): 0.158

[INFO] Processing band 2...
[INFO] Loaded band 2 from D:\Yuvraj\MSI\bbr\data\200006033301_01\200006033301_01\200006033301_01_P001_MUL\24SEP27155103-M2AS-200006033301_01_P001_simulated_translations.tif, shape=(5102, 5189), dtype=uint8
[INFO] SIFT detector initialized
[INFO] Dividing image into 5 x 5 blocks of approx 1020 x 1037 pixels
[INFO] Feature extraction completed
runnning optimised version for ransac on self.device='cuda'
[INFO] Band 2 - Number of inliers: 130
[INFO] Band 2 with ref band - RMSE in pixels): 0.120

[INFO] Processing band 4...
[INFO] Loaded band 4 from D:\Yuvraj\MSI\bbr\data\200006033301_01\200006033301_01\200006033301_01_P001_MUL\24SEP27155103-M2AS-200006033301_01_P001_simulated_translations.tif, shape=(5102, 5189), dtype=uint8
[INFO] SIFT detector initialized
[INFO] Dividing image into 5 x 5 blocks of approx 1020 x

In [ ]:
# Simulation: translations + Rotations

In [1]:
import numpy as np
import rasterio
from skimage.transform import ProjectiveTransform, warp
from skimage.transform import AffineTransform, warp
from extract_sift_features import extract_sift_features
from sklearn.linear_model import RANSACRegressor
from sklearn.metrics import mean_squared_error
import os
import time
start_time = time.time()


# -------------------------
# INPUT PARAMETERS
# -------------------------
image_patch = r"D:\Yuvraj\MSI\bbr\data\200006033301_01\200006033301_01\200006033301_01_P001_MUL\24SEP27155103-M2AS-200006033301_01_P001_simulated_trans_and_rotation.tif"
output_path = r"D:\Yuvraj\MSI\bbr\data\200006033301_01\200006033301_01\200006033301_01_P001_MUL\simulated_rotations_registered_stack_.tif"
ref_band = 3

# SIFT parameters
sift_params = {
    "image_row": 10,
    "image_col": 10,
    "nfeatures": 100,
    "nOctaveLayers": 3,
    "contrastThreshold": 0.01,
    "edgeThreshold": 15,
    "sigma": 1.2,
    "eightbit_stretch": False,
    "sobel_filter": False,
    "return_keypoints": True,
    "return_descriptors": True,
    "return_coords": True,
    "display_figure": False
}

# -------------------------
# Load image and metadata
# -------------------------
with rasterio.open(image_patch) as src:
    image_data = src.read()  # shape: (bands, H, W)
    image_profile = src.profile
    total_bands = src.count
    input_dtype = src.dtypes[0]

print(f"[INFO] Total bands: {total_bands}, dtype: {input_dtype}")

# Initialize band stack with reference band
band_stack = np.zeros_like(image_data, dtype=np.float32)
band_stack[ref_band - 1] = image_data[ref_band - 1]

# Get other bands dynamically
other_bands = [b for b in range(1, total_bands + 1) if b != ref_band]

# -------------------------
# Step 1: Extract features from reference band
# -------------------------
print(f"[INFO] Extracting features from reference band {ref_band}...")
block_info_ref, all_kps_ref, all_desc_ref, all_coords_ref = extract_sift_features(
    image_patch, band_number=ref_band, **sift_params
)
print(f"[INFO] Reference band features extracted. Blocks: {len(block_info_ref)}")

# -------------------------
# Helper function: Align keypoints block-wise
# -------------------------
def align_block_keypoints(block_info_image1, block_info_image2):
    strong_points_image1_sorted = []
    strong_points_image2_sorted = []
    strong_points_image1_sorted_response = []
    strong_points_image2_sorted_response = []

    strong_block_info_image1 = {}
    strong_block_info_image2 = {}

    for block_number in block_info_image1:
        if block_number in block_info_image2:
            if block_info_image1[block_number]['status'] == 1 and block_info_image2[block_number]['status'] == 1:
                num1 = block_info_image1[block_number]['num_points']
                num2 = block_info_image2[block_number]['num_points']
                min_points = min(num1, num2)

                # Take top points by response
                kps1_sorted = sorted(block_info_image1[block_number]['keypoints'], key=lambda x: x['response'], reverse=True)[:min_points]
                kps2_sorted = sorted(block_info_image2[block_number]['keypoints'], key=lambda x: x['response'], reverse=True)[:min_points]

                strong_block_info_image1[block_number] = {'block_number': block_number, 'status': 1, 'num_points': min_points, 'keypoints': kps1_sorted}
                strong_block_info_image2[block_number] = {'block_number': block_number, 'status': 1, 'num_points': min_points, 'keypoints': kps2_sorted}

                for kp in kps1_sorted:
                    strong_points_image1_sorted.append(kp['coord'])
                    strong_points_image1_sorted_response.append(kp['response'])
                for kp in kps2_sorted:
                    strong_points_image2_sorted.append(kp['coord'])
                    strong_points_image2_sorted_response.append(kp['response'])

    return (
        np.array(strong_points_image1_sorted),
        np.array(strong_points_image2_sorted),
        np.array(strong_points_image1_sorted_response),
        np.array(strong_points_image2_sorted_response),
        strong_block_info_image1,
        strong_block_info_image2
    )

# -------------------------
# Step 2: Process and register all other bands
# -------------------------
for band in other_bands:
    print(f"\n[INFO] Processing band {band}...")

    # Extract features for this band
    block_info_band, all_kps_band, all_desc_band, all_coords_band = extract_sift_features(
        image_patch, band_number=band, **sift_params
    )

    # Align keypoints block-wise
    strong_pts_ref, strong_pts_band, resp_ref, resp_band, strong_block_ref, strong_block_band = align_block_keypoints(block_info_ref, block_info_band)
    print(f"[INFO] Band {band}_ - Total candidate matches: {len(strong_pts_ref)}")



    import sys
    #sys.path.append("utils")  # or full path to folder
    from ransacgpu import RANSACRegressorGPU

    # --------------------------------------------------
    # MSAC
    # --------------------------------------------------

    # Define the MSAC-based RANSAC model

    msac_model = RANSACRegressorGPU(
        min_samples=3,
        residual_threshold=19,  # Adjusted to handle expected noise scale
        max_trials=80000,  # Increased for robustness in large datasets
        stop_n_inliers=len(strong_pts_ref),
        loss='absolute_error',  # Robust against potential outliers
        random_state=42,  # Ensures reproducibility
    )
    # Fit the model
    msac_model.fit(strong_pts_ref, strong_pts_band)
    #predicted_pts_band = msac_model.predict(strong_pts_ref)
    inlier_mask = msac_model.inlier_mask_
    outlier_mask = ~inlier_mask

    inliers_ref = strong_pts_ref[inlier_mask]
    inliers_band = strong_pts_band[inlier_mask]
    inliers_ref_resp = resp_ref[inlier_mask]
    inliers_band_resp = resp_band[inlier_mask]

    # RMSE for inliers
    rmse = np.sqrt(mean_squared_error(inliers_ref, inliers_band))
    #print(f"[INFO] Band {band} - Number of inliers: {len(inliers_ref)}, RMSE: {rmse:.3f}")
    print(f"[INFO] Band {band} - Number of inliers: {len(inliers_ref)}")


    # Estimate affine transform (warp this band to reference)
    affine_tf = AffineTransform()
    affine_tf.estimate(inliers_band, inliers_ref)  # Warp band → ref

    # Print affine transformation matrix in a readable form
    print("[INFO] Affine Transformation Matrix (band → reference):")
    for row in affine_tf.params:
        print("  ".join(f"{val:.8f}" for val in row))
    # Decompose
    from affine_utils import decompose_affine
    params = decompose_affine(affine_tf)
    print(params)  # You can also access dict values

    # RMSE after affine transform
    warped_pts = affine_tf(inliers_band)
    rmse_transformed = np.sqrt(mean_squared_error(warped_pts, inliers_ref))
    print(f"[INFO] Band {band} - RMSE after affine transform: {rmse_transformed:.3f}")

    # Warp the band to reference
    warped_band = warp(image_data[band-1], inverse_map=affine_tf.inverse, output_shape=image_data[0].shape, order=3)
    band_stack[band-1] = warped_band.astype(np.float32)


# -------------------------
# Step 3: Export band_stack as GeoTIFF
# -------------------------
# Update metadata
image_profile.update({
    "count": total_bands,
    "dtype": np.float32
})

with rasterio.open(output_path, "w", **image_profile) as dst:
    dst.write(band_stack)

print(f"\n✅ Registered band stack saved at: {output_path}")
#########################
end_time = time.time()
elapsed = end_time - start_time
# Convert to minutes and seconds
mins, secs = divmod(elapsed, 60)
print(f"⏱️ Process completed in {int(mins)} min {secs:.2f} sec")

[INFO] Total bands: 8, dtype: uint8
[INFO] Extracting features from reference band 3...
[INFO] Loaded band 3 from D:\Yuvraj\MSI\bbr\data\200006033301_01\200006033301_01\200006033301_01_P001_MUL\24SEP27155103-M2AS-200006033301_01_P001_simulated_trans_and_rotation.tif, shape=(5102, 5189), dtype=uint8
[INFO] SIFT detector initialized
[INFO] Dividing image into 10 x 10 blocks of approx 510 x 518 pixels
[INFO] Feature extraction completed
[INFO] Reference band features extracted. Blocks: 100

[INFO] Processing band 1...
[INFO] Loaded band 1 from D:\Yuvraj\MSI\bbr\data\200006033301_01\200006033301_01\200006033301_01_P001_MUL\24SEP27155103-M2AS-200006033301_01_P001_simulated_trans_and_rotation.tif, shape=(5102, 5189), dtype=uint8
[INFO] SIFT detector initialized
[INFO] Dividing image into 10 x 10 blocks of approx 510 x 518 pixels
[INFO] Feature extraction completed
[INFO] Band 1_ - Total candidate matches: 10000
Package Version: 2.0.0a0


2025/11/05 13:07:45 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.


runnning optimised version for ransac on self.device='cuda'
[INFO] Band 1 - Number of inliers: 305
[INFO] Affine Transformation Matrix (band → reference):
1.00005385  -0.00881415  142.30594196
0.00869411  0.99975201  -81.84391363
0.00000000  0.00000000  1.00000000
[INFO] Affine Transformation Matrix:
[[1.00005385 -0.00881415 142.30594196]
 [0.00869411 0.99975201 -81.84391363]
 [0.00000000 0.00000000 1.00000000]]
[INFO] Decomposed Parameters:
  Translation : (tx=142.30594196, ty=-81.84391363)
  Rotation    : 0.49809667°
  Scale       : (sx=1.00009164, sy=0.99979086)
  Shear       : 0.00703034°
{'translation': (np.float64(142.30594196193397), np.float64(-81.84391363368468)), 'rotation_deg': np.float64(0.4980966650714717), 'scale': (np.float64(1.0000916430562263), np.float64(0.9997908594872672)), 'shear_deg': np.float64(0.007030336859795815)}
[INFO] Band 1 - RMSE after affine transform: 3.896

[INFO] Processing band 2...
[INFO] Loaded band 2 from D:\Yuvraj\MSI\bbr\data\200006033301_01\200

In [ ]:
# Rmse calculation: sim translations + rotations

In [1]:
import numpy as np
import rasterio
from skimage.transform import ProjectiveTransform, warp
from skimage.transform import AffineTransform, warp
from extract_sift_features import extract_sift_features
from sklearn.linear_model import RANSACRegressor
from sklearn.metrics import mean_squared_error
import os
import time
start_time = time.time()


# -------------------------
# INPUT PARAMETERS
# -------------------------
image_patch = r"D:\Yuvraj\MSI\bbr\data\200006033301_01\200006033301_01\200006033301_01_P001_MUL\24SEP27155103-M2AS-200006033301_01_P001_simulated_trans_and_rotation.tif"
output_path = r"D:\Yuvraj\MSI\bbr\data\200006033301_01\200006033301_01\200006033301_01_P001_MUL\simulated_rotations_registered_stack_.tif"
ref_band = 3

# SIFT parameters
sift_params = {
    "image_row": 5,
    "image_col": 5,
    "nfeatures": 100,
    "nOctaveLayers": 3,
    "contrastThreshold": 0.01,
    "edgeThreshold": 15,
    "sigma": 1.2,
    "eightbit_stretch": False,
    "sobel_filter": False,
    "return_keypoints": True,
    "return_descriptors": True,
    "return_coords": True,
    "display_figure": False
}

# -------------------------
# Load image and metadata
# -------------------------
with rasterio.open(image_patch) as src:
    image_data = src.read()  # shape: (bands, H, W)
    image_profile = src.profile
    total_bands = src.count
    input_dtype = src.dtypes[0]

print(f"[INFO] Total bands: {total_bands}, dtype: {input_dtype}")

# Initialize band stack with reference band
band_stack = np.zeros_like(image_data, dtype=np.float32)
band_stack[ref_band - 1] = image_data[ref_band - 1]

# Get other bands dynamically
other_bands = [b for b in range(1, total_bands + 1) if b != ref_band]

# -------------------------
# Step 1: Extract features from reference band
# -------------------------
print(f"[INFO] Extracting features from reference band {ref_band}...")
block_info_ref, all_kps_ref, all_desc_ref, all_coords_ref = extract_sift_features(
    image_patch, band_number=ref_band, **sift_params
)
print(f"[INFO] Reference band features extracted. Blocks: {len(block_info_ref)}")

# -------------------------
# Helper function: Align keypoints block-wise
# -------------------------
def align_block_keypoints(block_info_image1, block_info_image2):
    strong_points_image1_sorted = []
    strong_points_image2_sorted = []
    strong_points_image1_sorted_response = []
    strong_points_image2_sorted_response = []

    strong_block_info_image1 = {}
    strong_block_info_image2 = {}

    for block_number in block_info_image1:
        if block_number in block_info_image2:
            if block_info_image1[block_number]['status'] == 1 and block_info_image2[block_number]['status'] == 1:
                num1 = block_info_image1[block_number]['num_points']
                num2 = block_info_image2[block_number]['num_points']
                min_points = min(num1, num2)

                # Take top points by response
                kps1_sorted = sorted(block_info_image1[block_number]['keypoints'], key=lambda x: x['response'], reverse=True)[:min_points]
                kps2_sorted = sorted(block_info_image2[block_number]['keypoints'], key=lambda x: x['response'], reverse=True)[:min_points]

                strong_block_info_image1[block_number] = {'block_number': block_number, 'status': 1, 'num_points': min_points, 'keypoints': kps1_sorted}
                strong_block_info_image2[block_number] = {'block_number': block_number, 'status': 1, 'num_points': min_points, 'keypoints': kps2_sorted}

                for kp in kps1_sorted:
                    strong_points_image1_sorted.append(kp['coord'])
                    strong_points_image1_sorted_response.append(kp['response'])
                for kp in kps2_sorted:
                    strong_points_image2_sorted.append(kp['coord'])
                    strong_points_image2_sorted_response.append(kp['response'])

    return (
        np.array(strong_points_image1_sorted),
        np.array(strong_points_image2_sorted),
        np.array(strong_points_image1_sorted_response),
        np.array(strong_points_image2_sorted_response),
        strong_block_info_image1,
        strong_block_info_image2
    )

# -------------------------
# Step 2: Process and register all other bands
# -------------------------
for band in other_bands:
    print(f"\n[INFO] Processing band {band}...")

    # Extract features for this band
    block_info_band, all_kps_band, all_desc_band, all_coords_band = extract_sift_features(
        image_patch, band_number=band, **sift_params
    )

    # Align keypoints block-wise
    strong_pts_ref, strong_pts_band, resp_ref, resp_band, strong_block_ref, strong_block_band = align_block_keypoints(block_info_ref, block_info_band)
    #print(f"[INFO] Band {band}_ - Total candidate matches: {len(strong_pts_ref)}")


    # # MSAC / RANSAC to get inliers
    # msac_model = RANSACRegressor(
    #     min_samples=3,
    #     residual_threshold=19,
    #     max_trials=80000,
    #     stop_n_inliers=len(strong_pts_ref),
    #     loss='absolute_error',
    #     random_state=42
    # )
    # msac_model.fit(strong_pts_ref, strong_pts_band)
    # predicted_pts_band = msac_model.predict(strong_pts_ref)
    # inlier_mask = msac_model.inlier_mask_
    # outlier_mask = ~inlier_mask

    # inliers_ref = strong_pts_ref[inlier_mask]
    # inliers_band = strong_pts_band[inlier_mask]
    # inliers_ref_resp = resp_ref[inlier_mask]
    # inliers_band_resp = resp_band[inlier_mask]

    import sys
    #sys.path.append("utils")  # or full path to folder
    from ransacgpu import RANSACRegressorGPU

    # --------------------------------------------------
    # MSAC
    # --------------------------------------------------

    # Define the MSAC-based RANSAC model

    msac_model = RANSACRegressorGPU(
        min_samples=3,
        residual_threshold=3,  # Adjusted to handle expected noise scale
        max_trials=10000,  # Increased for robustness in large datasets
        stop_n_inliers=len(strong_pts_ref),
        loss='absolute_error',  # Robust against potential outliers
        random_state=42,  # Ensures reproducibility
    )
    # Fit the model
    msac_model.fit(strong_pts_ref, strong_pts_band)
    #predicted_pts_band = msac_model.predict(strong_pts_ref)
    inlier_mask = msac_model.inlier_mask_
    outlier_mask = ~inlier_mask

    inliers_ref = strong_pts_ref[inlier_mask]
    inliers_band = strong_pts_band[inlier_mask]
    inliers_ref_resp = resp_ref[inlier_mask]
    inliers_band_resp = resp_band[inlier_mask]

    # RMSE for inliers
    rmse = np.sqrt(mean_squared_error(inliers_ref, inliers_band))
    #print(f"[INFO] Band {band} - Number of inliers: {len(inliers_ref)}, RMSE: {rmse:.3f}")
    print(f"[INFO] Band {band} - Number of inliers: {len(inliers_ref)}")

    # # Estimate projective transform (warp this band to reference)
    # projective_tf = ProjectiveTransform()
    # projective_tf.estimate(inliers_band, inliers_ref)  # Warp band → ref

    # # RMSE after transform
    # warped_pts = projective_tf(inliers_band)
    # rmse_transformed = np.sqrt(mean_squared_error(warped_pts, inliers_ref))
    # print(f"[INFO] Band {band} - RMSE after projective transform: {rmse_transformed:.3f}")

    # # Warp the band to reference
    # warped_band = warp(image_data[band-1], projective_tf.inverse, output_shape=image_data[0].shape, order=3)
    # band_stack[band-1] = warped_band.astype(np.float32)

    # Estimate affine transform (warp this band to reference)
    affine_tf = AffineTransform()
    affine_tf.estimate(inliers_band, inliers_ref)  # Warp band → ref

    # # Print affine transformation matrix in a readable form
    # print("[INFO] Affine Transformation Matrix (band → reference):")
    # for row in affine_tf.params:
    #     print("  ".join(f"{val:.8f}" for val in row))
    # # Decompose
    # from affine_utils import decompose_affine
    # params = decompose_affine(affine_tf)
    # print(params)  # You can also access dict values

# # --- 1. RMSE on all points (before transform) ---
#     rmse_all_raw = np.sqrt(mean_squared_error(strong_pts_band, strong_pts_ref))
#     rmse_x_raw = np.sqrt(np.mean((strong_pts_band[:,0] - strong_pts_ref[:,0])**2))
#     rmse_y_raw = np.sqrt(np.mean((strong_pts_band[:,1] - strong_pts_ref[:,1])**2))

    # print(f"[INFO] Band {band} - RMSE all points (raw, 2D): {rmse_all_raw:.3f}")
    # print(f"[INFO] Band {band} - RMSE all points (raw, X):  {rmse_x_raw:.3f}")
    # print(f"[INFO] Band {band} - RMSE all points (raw, Y):  {rmse_y_raw:.3f}")

    # 2. RMSE on affine-transformed inliers
    warped_inliers = affine_tf(inliers_band)
    rmse_inliers = np.sqrt(mean_squared_error(warped_inliers, inliers_ref))
    print(f"[INFO] Band {band} with ref band - RMSE in pixels): {rmse_inliers:.3f}")

    # # 3. RMSE on affine-transformed all points
    # warped_all = affine_tf(strong_pts_band)
    # rmse_all_transformed = np.sqrt(mean_squared_error(warped_all, strong_pts_ref))
    # print(f"[INFO] Band {band} - RMSE on affine-transformed all points: {rmse_all_transformed:.3f}")

    # # Warp the band to reference
    # warped_band = warp(image_data[band-1], inverse_map=affine_tf.inverse, output_shape=image_data[0].shape, order=3)
    # band_stack[band-1] = warped_band.astype(np.float32)


# # -------------------------
# # Step 3: Export band_stack as GeoTIFF
# # -------------------------
# # Update metadata
# image_profile.update({
#     "count": total_bands,
#     "dtype": np.float32
# })

# with rasterio.open(output_path, "w", **image_profile) as dst:
#     dst.write(band_stack)

# print(f"\n✅ Registered band stack saved at: {output_path}")
#########################
end_time = time.time()
elapsed = end_time - start_time
# Convert to minutes and seconds
mins, secs = divmod(elapsed, 60)
print(f"⏱️ Process completed in {int(mins)} min {secs:.2f} sec")

[INFO] Total bands: 8, dtype: uint8
[INFO] Extracting features from reference band 3...
[INFO] Loaded band 3 from D:\Yuvraj\MSI\bbr\data\200006033301_01\200006033301_01\200006033301_01_P001_MUL\24SEP27155103-M2AS-200006033301_01_P001_simulated_trans_and_rotation.tif, shape=(5102, 5189), dtype=uint8
[INFO] SIFT detector initialized
[INFO] Dividing image into 5 x 5 blocks of approx 1020 x 1037 pixels
[INFO] Feature extraction completed
[INFO] Reference band features extracted. Blocks: 25

[INFO] Processing band 1...
[INFO] Loaded band 1 from D:\Yuvraj\MSI\bbr\data\200006033301_01\200006033301_01\200006033301_01_P001_MUL\24SEP27155103-M2AS-200006033301_01_P001_simulated_trans_and_rotation.tif, shape=(5102, 5189), dtype=uint8
[INFO] SIFT detector initialized
[INFO] Dividing image into 5 x 5 blocks of approx 1020 x 1037 pixels
[INFO] Feature extraction completed
Package Version: 2.0.0a0


2025/11/05 13:10:02 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.


runnning optimised version for ransac on self.device='cuda'
[INFO] Band 1 - Number of inliers: 78
[INFO] Band 1 with ref band - RMSE in pixels): 0.363

[INFO] Processing band 2...
[INFO] Loaded band 2 from D:\Yuvraj\MSI\bbr\data\200006033301_01\200006033301_01\200006033301_01_P001_MUL\24SEP27155103-M2AS-200006033301_01_P001_simulated_trans_and_rotation.tif, shape=(5102, 5189), dtype=uint8
[INFO] SIFT detector initialized
[INFO] Dividing image into 5 x 5 blocks of approx 1020 x 1037 pixels
[INFO] Feature extraction completed
runnning optimised version for ransac on self.device='cuda'
[INFO] Band 2 - Number of inliers: 15
[INFO] Band 2 with ref band - RMSE in pixels): 0.827

[INFO] Processing band 4...
[INFO] Loaded band 4 from D:\Yuvraj\MSI\bbr\data\200006033301_01\200006033301_01\200006033301_01_P001_MUL\24SEP27155103-M2AS-200006033301_01_P001_simulated_trans_and_rotation.tif, shape=(5102, 5189), dtype=uint8
[INFO] SIFT detector initialized
[INFO] Dividing image into 5 x 5 blocks of ap